In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/movie-review
/kaggle/input/movie-review/movie_reviews
/kaggle/input/movie-review/movie_reviews/movie_reviews
/kaggle/input/movie-review/movie_reviews/movie_reviews/pos
/kaggle/input/movie-review/movie_reviews/movie_reviews/neg


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import random

# Charger le dataset

positive_reviews_path = "/kaggle/input/movie-review/movie_reviews/movie_reviews/pos/"
negative_reviews_path = "/kaggle/input/movie-review/movie_reviews/movie_reviews/neg/"

def load_reviews_from_path(path, label):
    whole_reviews = []
    for filename in os.listdir(path):
        with open(os.path.join(path, filename)) as file:
            content = file.read()
            whole_reviews.append({'Text': content, 'Tag': label})
            
    return whole_reviews

# Load positive and negative reviews
positive_reviews = load_reviews_from_path(positive_reviews_path, 'positive')
negative_reviews = load_reviews_from_path(negative_reviews_path, 'negative')

cumulate_reviews = positive_reviews + negative_reviews

# Shuffle the combined dataset randomly
random.shuffle(cumulate_reviews)

df = pd.DataFrame(cumulate_reviews)

# Preprocessing des données textuelles
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

df['Text'] = df['Text'].apply(preprocess_text)

# Entraînement du modèle Word2Vec
model = Word2Vec(sentences=df['Text'], vector_size=100, window=5, min_count=1, workers=4)

# Vectorisation des reviews de movies
def vectorize_review(review):
    word_vectors = [model.wv[word] for word in review if word in model.wv]
    if word_vectors:
        return sum(word_vectors) / len(word_vectors)
    else:
        return [0] * model.vector_size

df['Vectorized_Text'] = df['Text'].apply(vectorize_review)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(list(df['Vectorized_Text']), df['Tag'], test_size=0.2, random_state=42)

# Construction d'un classificateur (exemple avec Logistic Regression)
classifier = LogisticRegression()
classifier.fit(list(X_train), y_train)

# Évaluation du modèle
y_pred = classifier.predict(list(X_test))

accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report_result)


Accuracy: 0.6225
Classification Report:
               precision    recall  f1-score   support

    negative       0.62      0.59      0.60       194
    positive       0.63      0.66      0.64       206

    accuracy                           0.62       400
   macro avg       0.62      0.62      0.62       400
weighted avg       0.62      0.62      0.62       400

